In [26]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import time

In [27]:
COLUMN_NAMES = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in',
    'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
    'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
    'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate',
    'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
    'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
    'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate',
    'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'class', 'difficulty'
]
CATEGORICAL_FEATURES = ['protocol_type', 'service', 'flag']
LABEL_COLUMN = 'class'
NORMAL_CLASS_NAME = 'normal'

def preprocess(df):
    X = df.drop(columns=[LABEL_COLUMN, 'difficulty'])
    y_labels = df[LABEL_COLUMN]
    
    y = np.where(y_labels == NORMAL_CLASS_NAME, 1, -1)
    
    X_dummies = pd.get_dummies(X, columns=CATEGORICAL_FEATURES, dummy_na=False)
        
    return X_dummies, y

print("Carregando e processando KDDTrain+.txt...")
try:
    df_train = pd.read_csv('KDDTrain+.txt', header=None, names=COLUMN_NAMES)
except FileNotFoundError:
    print("ERRO: Arquivo 'KDDTrain+.txt' não encontrado.")
    exit()

X_train_df, y_train = preprocess(df_train)

Carregando e processando KDDTrain+.txt...


In [17]:
print(f"Carregando dados de {FILE_PATH}...")
start_time = time.time()

try:
  df = pd.read_csv(FILE_PATH, nrows=NROWS_FOR_TESTING)
except FileNotFoundError:
  print(f"ERRO: Arquivo não encontrado em '{FILE_PATH}'.")
  exit()
except Exception as e:
  print(f"Erro ao ler o CSV: {e}")
  exit()

print(f"Dados carregados em {time.time() - start_time:.2f} segundos.")
print(f"Total de linhas carregadas: {len(df)}")
print(f"\nDistribuição das classes (coluna '{LABEL_COLUMN}'):\n{df[LABEL_COLUMN].value_counts()}")

Carregando dados de data_1.csv...
Dados carregados em 0.37 segundos.
Total de linhas carregadas: 200000

Distribuição das classes (coluna 'attack'):
attack
1    198779
0      1221
Name: count, dtype: int64


/tmp/ipykernel_69844/691213724.py:5: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(FILE_PATH, nrows=NROWS_FOR_TESTING)


In [28]:
print("\nCarregando e processando KDDTest+.txt...")
try:
    df_test = pd.read_csv('KDDTest+.txt', header=None, names=COLUMN_NAMES)
except FileNotFoundError:
    print("ERRO: Arquivo 'KDDTest+.txt' não encontrado.")
    exit()

X_test_df, y_test = preprocess(df_test)

print("\nAlinhando colunas de treino e teste...")

train_cols = X_train_df.columns

X_test_aligned = X_test_df.reindex(columns=train_cols, fill_value=0)

print(f"Formato Treino: {X_train_df.shape}, Formato Teste Alinhado: {X_test_aligned.shape}")
if X_train_df.shape[1] != X_test_aligned.shape[1]:
     print("ERRO: Alinhamento falhou!")
     exit()

print("Escalonando dados...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_df)
X_test_scaled = scaler.transform(X_test_aligned) 

print("\nTreinando o modelo IsolationForest...")
start_time = time.time()

contamination_rate = (y_train == -1).sum() / len(y_train)
print(f"Taxa de contaminação (anomalias) no treino: {contamination_rate:.4f}")


Carregando e processando KDDTest+.txt...

Alinhando colunas de treino e teste...
Formato Treino: (125973, 122), Formato Teste Alinhado: (22544, 122)
Escalonando dados...

Treinando o modelo IsolationForest...
Taxa de contaminação (anomalias) no treino: 0.4654


In [29]:
model = IsolationForest(
    n_estimators=100,
    contamination=contamination_rate,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_scaled) 
print(f"Treinamento concluído em {time.time() - start_time:.2f} segundos.")

print("\nAvaliando o modelo no conjunto de TESTE...")
y_pred = model.predict(X_test_scaled) 

Treinamento concluído em 20.81 segundos.

Avaliando o modelo no conjunto de TESTE...


In [23]:
# --- 5. !! NOVA ESTRATÉGIA: UNDERSAMPLING PARA O TREINO !! ---
print("\nCriando conjunto de treino com Undersampling...")

# 5.1. Separar dados de treino por classe
X_train_normal = X_train[y_train == 1]
y_train_normal = y_train[y_train == 1]
X_train_anomaly = X_train[y_train == -1]
y_train_anomaly = y_train[y_train == -1]

print(f"Amostras de treino 'Normal': {len(X_train_normal)}")
print(f"Amostras de treino 'Anomalia': {len(X_train_anomaly)}")

# 5.2. Definir o tamanho da amostra de anomalias
# Vamos pegar um número bem menor de anomalias, mas ainda dominante
# Ex: 30.000 (você pode ajustar esse número)
N_ANOMALY_SAMPLES = 30000 
if N_ANOMALY_SAMPLES > len(X_train_anomaly):
    N_ANOMALY_SAMPLES = len(X_train_anomaly) # Caso tenha menos

# 5.3. Fazer a amostragem aleatória (sem reposição)
np.random.seed(42) # para reprodutibilidade
indices = np.random.choice(len(X_train_anomaly), N_ANOMALY_SAMPLES, replace=False)
X_train_anomaly_sampled = X_train_anomaly[indices]
y_train_anomaly_sampled = y_train_anomaly[indices]

# 5.4. Criar o novo conjunto de treino (balanceado)
X_train_new = np.concatenate([X_train_normal, X_train_anomaly_sampled])
y_train_new = np.concatenate([y_train_normal, y_train_anomaly_sampled])

print(f"Novo conjunto de treino: {len(X_train_new)} amostras")

# 5.5. Calcular a *nova* taxa de contaminação para este conjunto
contamination_rate = (y_train_new == -1).sum() / len(y_train_new)
print(f"Taxa de contaminação NO TREINO: {contamination_rate:.4f}")


Criando conjunto de treino com Undersampling...
Amostras de treino 'Normal': 855
Amostras de treino 'Anomalia': 139145
Novo conjunto de treino: 30855 amostras
Taxa de contaminação NO TREINO: 0.9723


In [24]:
# --- 5. TREINAR O ISOLATION FOREST ---
print("\nTreinando o modelo IsolationForest...")
start_time = time.time()

model = IsolationForest(
  n_estimators=100,
  contamination='auto',
  random_state=42,
  n_jobs=-1
)

model.fit(X_train_new)

print(f"Treinamento concluído em {time.time() - start_time:.2f} segundos.")


Treinando o modelo IsolationForest...
Treinamento concluído em 0.43 segundos.


In [30]:
print("\nRelatório de Classificação (NSL-KDD):")
print(classification_report(y_test, y_pred, target_names=['Anomalia (-1)', 'Normal (1)']))
print("\nMatriz de Confusão (NSL-KDD):")
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("Formato: [ [Verdadeiro Anomalia, Falso Normal], [Falso Anomalia, Verdadeiro Normal] ]")


Relatório de Classificação (NSL-KDD):
               precision    recall  f1-score   support

Anomalia (-1)       0.81      0.82      0.81     12833
   Normal (1)       0.76      0.74      0.75      9711

     accuracy                           0.79     22544
    macro avg       0.78      0.78      0.78     22544
 weighted avg       0.79      0.79      0.79     22544


Matriz de Confusão (NSL-KDD):
[[10533  2300]
 [ 2532  7179]]
Formato: [ [Verdadeiro Anomalia, Falso Normal], [Falso Anomalia, Verdadeiro Normal] ]


In [ ]:
# --- 9. SALVAR O MODELO ---
print("\nSalvando o modelo treinado...")
joblib.dump(model, 'iforest_nslkdd_model.joblib')
joblib.dump(scaler, 'data_scaler_nslkdd.joblib')
print("Modelo salvo como 'iforest_nslkdd_model.joblib'")
print("\nScript concluído!")


Salvando o modelo treinado...
Modelo salvo como 'iforest_detector_model.joblib'
Scaler salvo como 'data_scaler.joblib'

Script concluído!
